In [1]:
from llm_providers import GoogleAIModel
import os
from dotenv import load_dotenv
from multi_agent import SyntheticDataGenerator
from langchain.retrievers import BM25Retriever
from utils import pdf_parser
from tasks import Task

load_dotenv()
key = os.environ.get('GOOGLE_API_KEY')
llm = GoogleAIModel(api_key=key)
pdf_files = pdf_parser('./test_pdf/book.pdf')

bm25retriever = BM25Retriever.from_documents(pdf_files, k=10)

agent = SyntheticDataGenerator(llm=llm, 
                               retriever=bm25retriever,
                               output_path='./output',
                               buffer_size=10)

example_task = Task(
    task_name="dpo",
    localization='Lora Finetuning and Adapters',
    task_description="I want to use your outputs to train a AI Researcher Model",
    rows_per_batch=12,
    batch_size=15,
    language="Vietnamese"
)

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Calling pdf_parser                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Extracted 31 pages. First page preview:                                                                         │
│ 1                                                                                                               │
│ LLM Post-Training: A Deep Dive into Reasoning                                                                   │
│ Large Language Models                                                                                           │
│ Komal Kumar∗, Tajamul Ashraf∗,...                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Loading 31 pieces of context!                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Split successful. Last chunk preview:                                                                           │
│ 31                                                                                                              │
│ progressonscalableoversightforlargelanguagemodels,” arXiv                                                       │
│ preprint arXiv:2211.03540, 2022. 20                                                                             │
│ [34...                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [4]:
import uuid

thread_config = {'configurable': {'thread_id': 123}}

async for output in agent.agent_flow.astream(
    {
        "task": example_task,
        "human_feedback": None,
        "conversations": []
    }
    ,
    config=thread_config, 
    stream_mode="updates"):
    for key, value in output.items():
        print(value)
    print("\n---\n")

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│ FORMATTED DOCUMENTS PREVIEW:                                                                                    │
│ - 13                                                                                                            │
│ Model Category Source Description                                                                               │
│ 1.Parameter-EfficientFine-Tuning&ModelCompression                                                               │
│ LoRA[60] Low-Rank Adaptation Link Injects trainable low-rank adapters for efficient fine-tuning.                │
│ QLoRA[188] Qu                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'retrieved_documents': ['\n- 13\nModel Category Source Description\n1.Parameter-EfficientFine-Tuning&ModelCompression\nLoRA[60] Low-Rank Adaptation Link Injects trainable low-rank adapters for efficient fine-tuning.\nQLoRA[188] Quantized Adaptation Link Combines 4-bit quantization with LoRA to enable fine-tuning on consumer GPUs\nGPTQ[189] Post-Training Quantization Link Optimal 4-bit quantization method for GPT-style models with minimal loss\nSparseGPT[190] Pruning Link One-shot pruning that preserves model quality with compensation.\nPEFT(HF) [191] Unified Fine-Tuning Link Library integrating LoRA, prefix tuning, and other parameter-efficient methods\nBitsAndBytes[192] Low-Precision Training Link Enables 8-bit optimizers and 4-bit quantization for memory-efficient training\nAdaLoRA[193] Adaptive Adaptation Link Dynamically allocates parameter budget between layers during fine-tuning\nP-Tuningv2 [194] Prompt Optimization Link Learns continuous prompt embeddings through deep prompt tu

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Total number of conversation turns up to this point: 0                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ Calling prompt_initialize                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── INFO ──────────────────────────────────────────────────────╮
│ System Prompt:                                                                                                  │
│ You are an advanced synthetic data generator, engineered to produce high-quality, task-specific syn             │
│ User Prompt: You are tasked to help me generate a dataset of 2 rows entirely in Vietnamese, based entirely on   │
│ the                                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ ---------FISH_FOR_FEEDBACK---------                                                                             │
│                                                                                                                 │
│ System Prompt:                                                                                                  │
│ You are an advanced synthetic data generator, engineered to produce high-quality, task-specific synthetic       │
│ datasets. Your mission is to generate data samples in formats that precisely adhere to the requirements         │
│ provided.                                                                                                       │
│                                                                                                                 │
│ User Prompt:You are tasked to help me generate a dataset of 2 rows entirely in Vietnamese, based entirely on    │
│ the following context:                                                                                          │
│ - 13                                                                                                            │
│ Model Category Source Description                                                                               │
│ 1.Parameter-EfficientFine-Tuning&ModelCompression                                                               │
│ LoRA[60] Low-Rank Adaptation Link Injects trainable low-rank adapters for efficient fine-tuning.                │
│ QLoRA[188] Quantized Adaptation Link Combines 4-bit quantization with LoRA to enable fine-tuning on consumer    │
│ GPUs                                                                                                            │
│ GPTQ[189] Post-Training Quantization Link Optimal 4-bit quantization method for GPT-style models with minimal   │
│ loss                                                                                                            │
│ SparseGPT[190] Pruning Link One-shot pruning that preserves model quality with compensation.                    │
│ PEFT(HF) [191] Unified Fine-Tuning Link Library integrating LoRA, prefix tuning, and other parameter-efficient  │
│ methods                                                                                                         │
│ BitsAndBytes[192] Low-Precision Training Link Enables 8-bit optimizers and 4-bit quantization for               │
│ memory-efficient training                                                                                       │
│ AdaLoRA[193] Adaptive Adaptation Link Dynamically allocates parameter budget between layers during fine-tuning  │
│ P-Tuningv2 [194] Prompt Optimization Link Learns continuous prompt embeddings through deep prompt tuning        │
│ 2.DataManagement&Preprocessing                                                                                  │
│ HFDatasets [195] Data Processing Link Unified API for 30k+ datasets with streaming, versioning, and             │
│ preprocessing                                                                                                   │
│ WebDataset[196] Data Streaming Link Efficient tar-based sharding format for petascale distributed training      │
│ DVC[197] Data Versioning Link Git-like version control for datasets and machine learning pipelines              │
│ ApacheArrow [198] Memory Format Link Language-agnostic columnar memory format for zero-copy data access         │
│ Zstandard[199] Compression Link High-speed compression algorithm for training data storage/transfer             │
│ Cleanlab[200] Data Quality Link Automatic detection of label errors and outliers in training datasets           │
│ 3.DistributedTraining&Optimization                    

╭──────────────────────────────────────────────── OUTPUT_MESSAGE ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ ---------FISH_FOR_FEEDBACK---------                                                                             │
│                                                                                                                 │
│ Response:```json                                                                                                │
│ [                                                                                                               │
│   {                                                                                                             │
│     "prompt": "Hãy giải thích ngắn gọn về LoRA và QLoRA trong lĩnh vực tinh chỉnh mô hình ngôn ngữ lớn (LLM).", │
│     "chosen": "LoRA (Low-Rank Adaptation) là một phương pháp tinh chỉnh hiệu quả tham số, bằng cách chèn các    │
│ adapter có thứ hạng thấp có thể huấn luyện được vào mô hình. QLoRA (Quantized Adaptation) kết hợp lượng tử hóa  │
│ 4-bit với LoRA, cho phép tinh chỉnh trên các GPU tiêu dùng với chi phí bộ nhớ thấp hơn đáng kể. Cả hai đều giúp │
│ giảm đáng kể lượng tài nguyên tính toán cần thiết để điều chỉnh LLM cho các tác vụ cụ thể.",                    │
│     "rejected": "LoRA và QLoRA là các thuật toán nén dữ liệu."                                                  │
│   },                                                                                                            │
│   {                                                                                                             │
│     "prompt": "Mô tả vai trò của HF Datasets và WebDataset trong quản lý và tiền xử lý dữ liệu cho huấn luyện   │
│ mô hình ngôn ngữ lớn.",                                                                                         │
│     "chosen": "HF Datasets cung cấp một API thống nhất cho hơn 30.000 bộ dữ liệu, hỗ trợ streaming, quản lý     │
│ phiên bản và tiền xử lý dữ liệu. WebDataset là một định dạng phân mảnh dựa trên tar hiệu quả, được thiết kế cho │
│ huấn luyện phân tán quy mô petabyte. HF Datasets giúp truy cập và quản lý dữ liệu dễ dàng hơn, trong khi        │
│ WebDataset tối ưu hóa hiệu suất đọc/ghi dữ liệu trong quá trình huấn luyện phân tán.",                          │
│     "rejected": "HF Datasets và WebDataset là các công cụ để tạo ra dữ liệu giả."                               │
│   }                                                                                                             │
│ ]                                                                                                               │
│ ```                                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'response': '```json\n[\n  {\n    "prompt": "Hãy giải thích ngắn gọn về LoRA và QLoRA trong lĩnh vực tinh chỉnh mô hình ngôn ngữ lớn (LLM).",\n    "chosen": "LoRA (Low-Rank Adaptation) là một phương pháp tinh chỉnh hiệu quả tham số, bằng cách chèn các adapter có thứ hạng thấp có thể huấn luyện được vào mô hình. QLoRA (Quantized Adaptation) kết hợp lượng tử hóa 4-bit với LoRA, cho phép tinh chỉnh trên các GPU tiêu dùng với chi phí bộ nhớ thấp hơn đáng kể. Cả hai đều giúp giảm đáng kể lượng tài nguyên tính toán cần thiết để điều chỉnh LLM cho các tác vụ cụ thể.",\n    "rejected": "LoRA và QLoRA là các thuật toán nén dữ liệu."\n  },\n  {\n    "prompt": "Mô tả vai trò của HF Datasets và WebDataset trong quản lý và tiền xử lý dữ liệu cho huấn luyện mô hình ngôn ngữ lớn.",\n    "chosen": "HF Datasets cung cấp một API thống nhất cho hơn 30.000 bộ dữ liệu, hỗ trợ streaming, quản lý phiên bản và tiền xử lý dữ liệu. WebDataset là một định dạng phân mảnh dựa trên tar hiệu quả, được thiết kế cho 

RuntimeError: Called get_config outside of a runnable context

In [ ]:
from langgraph.types import Command

agent.agent_flow.ainvoke(
    Command(resume='yes'),
    config=thread_config
)